In [ ]:
import os

import numpy as np

import wurlitzer
from astropy import units as u

import matplotlib.pyplot as plt

import scipy
from scipy import optimize



In [ ]:
test_freq = [
    np.array([
        0.0610297, 0.1220594, 0.1830891, 0.2441188, 0.3051485, 0.3661782,
        0.4272079, 0.4882376, 0.57978215, 0.6713267, 0.73235641, 0.82390096,
        0.94596036, 1.06801976, 1.19007916, 1.34265341, 1.52574251, 1.70883161,
        1.92243556, 2.19706922, 2.50221772, 2.80736622, 3.14302957, 3.53972262,
        3.99744538, 4.51619783, 5.12649484, 5.79782154, 6.53017795, 7.3540789,
        8.30003926, 9.36805901, 10.55813817, 11.93130643, 13.48756379, 15.22691026,
        17.17986067, 19.37692988, 21.84863275, 24.65599897, 27.82954339, 31.43029572,
        35.4887708, 40.03548348, 45.16197831, 50.95979985, 57.52049265
    ]),
    np.array([
        0.10005003, 0.15007504, 0.20010005, 0.25012506, 0.30015008, 0.35017509,
        0.4002001, 0.45022511, 0.50025013, 0.57528764, 0.65032516, 0.72536268,
        0.82541271, 0.92546273, 1.02551276, 1.15057529, 1.30065033, 1.47573787,
        1.67583792, 1.87593797, 2.10105053, 2.37618809, 2.67633817, 3.02651326,
        3.42671336, 3.85192596, 4.32716358, 4.87743872, 5.50275138, 6.20310155,
        7.00350175, 7.87893947, 8.85442721, 9.97998999, 11.25562781, 12.68134067,
        14.28214107, 16.10805403, 18.15907954, 20.46023012, 23.06153077, 25.987994,
        29.28964482, 32.99149575, 37.16858429, 41.89594797, 47.1985993
    ])
]

test_psd = [
    np.array([
        6.86488638e+05, 1.23862278e+06, 9.26303299e+05, 3.60721675e+05,
        1.63842724e+05, 1.26530158e+05, 1.00421640e+05, 6.82068600e+04,
        2.88951257e+04, 1.24890704e+04, 6.66632965e+03, 1.92539118e+03,
        2.00925747e+02, 1.83700849e+02, 3.63300537e+02, 6.50880497e+02,
        5.77821197e+02, 2.56325420e+02, 3.37924769e+01, 2.50135085e+01,
        6.67960884e+01, 2.61745286e+01, 4.42448813e+00, 1.63095757e+01,
        3.90120276e+00, 5.71014135e+00, 1.73051847e+00, 2.44029925e+00,
        1.26352827e+00, 7.20185443e-01, 5.10883413e-01, 3.67615781e-01,
        2.45368711e-01, 1.78405279e-01, 1.42542399e-01, 1.04279944e-01,
        7.37068316e-02, 5.71334305e-02, 4.65681100e-02, 3.55492571e-02,
        2.90944455e-02, 2.37740527e-02, 1.97769160e-02, 1.68633739e-02,
        1.46859944e-02, 1.32646366e-02, 1.24911487e-02,
    ]),
    np.array([
        0.11589497, 0.12278325, 0.07503257, 0.04684548, 0.04663187, 0.0444642,
        0.03609387, 0.0264311,  0.02011643, 0.01928994, 0.02383489, 0.02487129,
        0.01966836, 0.01863879, 0.01781315, 0.01567394, 0.01289552, 0.01161635,
        0.00844275, 0.00942161, 0.00850452, 0.00756728, 0.00720028, 0.00612499,
        0.00716753, 0.00624518, 0.00550925, 0.00429895, 0.00512276, 0.00497993,
        0.00439096, 0.00398326, 0.00376943, 0.00372136, 0.00347776, 0.00307749,
        0.00342084, 0.00339543, 0.00319416, 0.00300875, 0.00314169, 0.00284596,
        0.00296397, 0.00277,    0.00281466, 0.00281973, 0.00274093,
    ])
]



In [ ]:
def estimate_net_func(x, a, b, c):
    return a * (x - b)**2 + c


def estimate_net(freqs, data):
    n_psd = len(data)
    offset = int(0.8 * n_psd)
    ffreq = np.log(freqs[offset:])
    fdata = np.log(data[offset:])
    
    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        ffreq,
        fdata,
        color="black",
        label="fit data",
    )
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("Val")
    ax.legend(loc="best")
    plt.show()
    
    params, params_cov = optimize.curve_fit(
        estimate_net_func, ffreq, fdata, p0=[1.0, ffreq[-1], fdata[-1]]
    )
    print(params)
    fdata = estimate_net_func(ffreq, params[0], params[1], params[2])
    fdata = np.exp(fdata)
    net = np.sqrt(fdata[-1])
    
    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(1, 1, 1)
    ax.loglog(
        freqs,
        data,
        color="black",
        label="Input PSD",
    )
    ax.loglog(
        freqs[offset:],
        fdata,
        color="red",
        label="Fit",
    )
    ax.axhline(
        net**2,
        label=f"NET = {net:.3f}",
        linestyle="--",
        color="blue",
    )
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("PSD [K$^2$ / Hz]")
    ax.legend(loc="best")
    plt.show()
    
    return net


In [ ]:
# fit_net = estimate_net(freq, psd)

In [ ]:
def evaluate_model(freqs, fmin, net, fknee, alpha):
    f_alpha = np.power(freqs, alpha)
    fknee_alpha = np.power(fknee, alpha)
    fmin_alpha = np.power(fmin, alpha)
    num = f_alpha + fknee_alpha
    den = f_alpha + fmin_alpha
    psd = (num / den) * net**2
    return psd

In [ ]:
# def fit_fun(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     data = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = kwargs["net"]
#     fknee = x[0]
#     alpha = x[1]
#     current = evaluate_model(freqs, fmin, net, fknee, alpha)
#     resid = current - data
#     return resid

# def fit_fun(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     data = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = x[0]
#     fknee = x[1]
#     alpha = x[2]
#     current = evaluate_model(freqs, fmin, net, fknee, alpha)

# #     weights = np.ones_like(current) * freqs[-1]
# #     weights -= freqs[-1] / (1.0 + freqs**2)
# #     resid = np.multiply(weights, current - data)
#     resid = current - data
#     return resid

In [ ]:
# def fit_jac(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     data = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = kwargs["net"]
#     fknee = x[0]
#     alpha = x[1]
#     n_freq = len(freqs)
    
#     f_alpha = np.power(freqs, alpha)
#     fknee_alpha = np.power(fknee, alpha)
#     fmin_alpha = np.power(fmin, alpha)
    
#     num = f_alpha + fknee_alpha
#     den = f_alpha + fmin_alpha
    
#     J = np.empty((n_freq, x.size), dtype=np.float64)
    
#     # Partial derivative wrt f_knee
#     J[:, 0] = (net**2 / den) * alpha * np.power(fknee, (alpha - 1))
    
#     # Partial derivative wrt alpha
#     da_num = f_alpha * np.log(freqs) + fknee_alpha * np.log(fknee)
#     da_den = f_alpha * np.log(freqs) + fmin_alpha * np.log(fmin)
#     J[:, 1] = (den * da_num - num * da_den) / den**2
    
#     return J


# def fit_jac(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     data = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = x[0]
#     fknee = x[1]
#     alpha = x[2]
#     n_freq = len(freqs)
    
#     f_alpha = np.power(freqs, alpha)
#     fknee_alpha = np.power(fknee, alpha)
#     fmin_alpha = np.power(fmin, alpha)
    
#     num = f_alpha + fknee_alpha
#     den = f_alpha + fmin_alpha
    
#     J = np.empty((n_freq, x.size), dtype=np.float64)
    
#     # Partial derivative wrt NET
#     J[:, 0] = 2.0 * net * (num / den)
    
#     # Partial derivative wrt f_knee
#     J[:, 1] = (net**2 / den) * alpha * np.power(fknee, (alpha - 1))
    
#     # Partial derivative wrt alpha
#     da_num = f_alpha * np.log(freqs) + fknee_alpha * np.log(fknee)
#     da_den = f_alpha * np.log(freqs) + fmin_alpha * np.log(fmin)
#     J[:, 2] = (den * da_num - num * da_den) / den**2
    
#     return J


In [ ]:
# def fit_psd(freqs, data):
#     fmin = 1.0e-5
#     fit_net = estimate_net(freqs, data)
#     midfreq = 0.5 * freqs[-1]
#     bounds = (
#         [freqs[0]/2.0, 0.1],
#         [freqs[-1], 5.0],
#     )
#     x_0 = np.array([midfreq, 1.0])
#     print(f"bounds = {bounds}")
#     print(f"guess = {x_0}")
#     result = optimize.least_squares(
#         fit_fun,
#         x_0,
#         jac=fit_jac,
#         bounds=bounds,
#         #loss='soft_l1',
#         #f_scale=0.1,
#         xtol=1.0e-12,
#         gtol=1.0e-12,
#         ftol=1.0e-12,
#         kwargs={"freqs": freqs, "data": data, "fmin": fmin, "net": fit_net},
#         verbose=1,
#     )
#     print(f"result solution = {result.x}")
#     print(f"result cost = {result.cost}")
#     print(f"result fun = {result.fun}")
#     print(f"result nfev = {result.nfev}")
#     print(f"result njev = {result.njev}")
#     print(f"result status = {result.status}")
#     print(f"result message = {result.message}")
#     print(f"result status = {result.status}")
#     print(f"result active_mask = {result.active_mask}")
#     if result.success:
#         return evaluate_model(freqs, fmin, fit_net, result.x[0], result.x[1])
#     else:
#         return data

In [ ]:
# def fit_minimize(x, *args):
#     net = x[0]
#     fknee = x[1]
#     alpha = x[2]
#     freqs = args[0]
#     data = args[1]
#     fmin = args[2]
#     current = evaluate_model(freqs, fmin, net, fknee, alpha)

# #     weights = np.ones_like(current) * freqs[-1]
# #     weights -= freqs[-1] / (1.0 + freqs**2)
# #     resid = np.multiply(weights, current - data)
#     resid = current - data
#     return np.sum(resid**2)

In [ ]:
# def psd_minimize(freqs, data):
#     fmin = 1.0e-5
#     plateau = np.sqrt(np.mean(data[-5:]))
#     midfreq = 0.5 * freqs[-1]
#     bounds = [
#         (0.5 * plateau, 2.0 * plateau),
#         (freqs[0]/2.0, freqs[-1]),
#         (0.1, 5.0),
#     ]
#     x_0 = np.array([plateau, midfreq, 1.0])
#     print(f"bounds = {bounds}")
#     print(f"guess = {x_0}")
#     result = optimize.shgo(
#         fit_minimize, 
#         bounds, 
#         args=(freqs, data, fmin),
#         minimizer_kwargs={"ftol": 1.0e-14},
#         iters=3,
#         n=5000,
#     )
#     print(f"result solution = {result.x}")
#     print(f"result fun = {result.fun}")
#     print(f"result message = {result.message}")
#     if result.success:
#         return evaluate_model(freqs, fmin, result.x[0], result.x[1], result.x[2])
#     else:
#         return data

In [ ]:
def evaluate_log_model(freqs, fmin, net, fknee, alpha):
    f_alpha = np.power(freqs, alpha)
    fknee_alpha = np.power(fknee, alpha)
    fmin_alpha = np.power(fmin, alpha)
    if net <= 0:
        print(f"error: log({net}) invalid")
    bad = (f_alpha + fknee_alpha) <= 0
    if np.count_nonzero(bad) > 0:
        print(f"error: knee alpha {fknee_alpha} log({f_alpha + fknee_alpha})")
    bad = (f_alpha + fmin_alpha) <= 0
    if np.count_nonzero(bad) > 0:
        print(f"error: min alpha {fmin_alpha} log({f_alpha + fmin_alpha})")
    #print(f"log model = 2.0*log({net}) + log({f_alpha}+{fknee_alpha}) - log({f_alpha}+{fmin_alpha})")
    psd = 2.0 * np.log(net) + np.log(f_alpha + fknee_alpha) - np.log(f_alpha + fmin_alpha)
    return psd

In [ ]:
def fit_log_fun(x, *args, **kwargs):
    freqs = kwargs["freqs"]
    logdata = kwargs["data"]
    fmin = kwargs["fmin"]
    net = kwargs["net"]
    fknee = x[0]
    alpha = x[1]
    current = evaluate_log_model(freqs, fmin, net, fknee, alpha)
    weights = np.ones_like(current) * freqs[-1]
    weights -= freqs[-1] / (1.0 + freqs**2)
    resid = np.multiply(weights, current - logdata)
    #resid = current - logdata
    return resid

# def fit_log_fun(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     logdata = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = x[0]
#     fknee = x[1]
#     alpha = x[2]
#     current = evaluate_log_model(freqs, fmin, net, fknee, alpha)
#     weights = np.ones_like(current) * freqs[-1]
#     weights -= freqs[-1] / (1.0 + freqs**2)
#     resid = np.multiply(weights, current - logdata)
#     #resid = current - logdata
#     return resid

In [ ]:
def fit_log_jac(x, *args, **kwargs):
    freqs = kwargs["freqs"]
    fmin = kwargs["fmin"]
    net = kwargs["net"]
    fknee = x[0]
    alpha = x[1]
    n_freq = len(freqs)
    
    log_freqs = np.log(freqs)
    f_alpha = np.power(freqs, alpha)
    fknee_alpha = np.power(fknee, alpha)
    fmin_alpha = np.power(fmin, alpha)
    
    fkalpha = f_alpha + fknee_alpha
    fmalpha = f_alpha + fmin_alpha
    
    J = np.empty((n_freq, x.size), dtype=np.float64)
    
    # Partial derivative wrt f_knee
    J[:, 0] = alpha * np.power(fknee, alpha-1.0) / fkalpha
    
    # Partial derivative wrt alpha
    J[:, 1] = (
        (f_alpha * log_freqs + fknee_alpha * np.log(fknee)) / fkalpha -
        (f_alpha * log_freqs + fmin_alpha * np.log(fmin)) / fmalpha
    )
    
    return J


# def fit_log_jac(x, *args, **kwargs):
#     freqs = kwargs["freqs"]
#     data = kwargs["data"]
#     fmin = kwargs["fmin"]
#     net = x[0]
#     fknee = x[1]
#     alpha = x[2]
#     n_freq = len(freqs)
    
#     log_freqs = np.log(freqs)
#     f_alpha = np.power(freqs, alpha)
#     fknee_alpha = np.power(fknee, alpha)
#     fmin_alpha = np.power(fmin, alpha)
    
#     fkalpha = f_alpha + fknee_alpha
#     fmalpha = f_alpha + fmin_alpha
    
#     J = np.empty((n_freq, x.size), dtype=np.float64)
    
#     # Partial derivative wrt NET
#     J[:, 0] = 2.0 / net
    
#     # Partial derivative wrt f_knee
#     J[:, 1] = alpha * np.power(fknee, alpha-1.0) / fkalpha
    
#     # Partial derivative wrt alpha
#     J[:, 2] = (
#         (f_alpha * log_freqs + fknee_alpha * np.log(fknee)) / fkalpha -
#         (f_alpha * log_freqs + fmin_alpha * np.log(fmin)) / fmalpha
#     )
    
#     return J


In [ ]:
def fit_log_psd(freqs, data):
    fmin = 1.0e-5
    net = estimate_net(freqs, data)
    
    log_data = np.log(data)
    print(f"log_data = {log_data}")

    midfreq = 0.5 * freqs[-1]
    bounds = (
        [freqs[0]/2.0, 0.1],
        [freqs[-1], 5.0],
    )
    x_0 = np.array([midfreq, 1.0])
    print(f"bounds = {bounds}")
    print(f"guess = {x_0}")
    result = optimize.least_squares(
        fit_log_fun,
        x_0,
        jac=fit_log_jac,
        bounds=bounds,
        #loss='soft_l1',
        #f_scale=0.1,
        xtol=1.0e-14,
        gtol=1.0e-14,
        ftol=1.0e-14,
        kwargs={"freqs": freqs, "data": log_data, "fmin": fmin, "net": net},
        verbose=1,
    )
    print(f"result solution = {result.x}")
    print(f"result cost = {result.cost}")
    print(f"result fun = {result.fun}")
    print(f"result nfev = {result.nfev}")
    print(f"result njev = {result.njev}")
    print(f"result status = {result.status}")
    print(f"result message = {result.message}")
    print(f"result status = {result.status}")
    print(f"result active_mask = {result.active_mask}")
    if result.success:
        return evaluate_model(freqs, fmin, net, result.x[0], result.x[1])
    else:
        return data


# def fit_log_psd(freqs, data):
#     fmin = 1.0e-5
#     log_data = np.log(data)
#     print(f"log_data = {log_data}")
#     plateau = np.mean(data[-5:])
#     guess_net = np.sqrt(plateau)
#     midfreq = 0.5 * freqs[-1]
#     bounds = (
#         [1.0e-16, freqs[0]/2.0, 0.1],
#         [np.inf, freqs[-1], 5.0],
#     )
#     x_0 = np.array([guess_net, midfreq, 1.0])
#     print(f"bounds = {bounds}")
#     print(f"guess = {x_0}")
#     result = optimize.least_squares(
#         fit_log_fun,
#         x_0,
#         jac=fit_log_jac,
#         bounds=bounds,
#         #loss='soft_l1',
#         #f_scale=0.1,
#         xtol=1.0e-14,
#         gtol=1.0e-14,
#         ftol=1.0e-14,
#         kwargs={"freqs": freqs, "data": log_data, "fmin": fmin},
#         verbose=1,
#     )
#     print(f"result solution = {result.x}")
#     print(f"result cost = {result.cost}")
#     print(f"result fun = {result.fun}")
#     print(f"result nfev = {result.nfev}")
#     print(f"result njev = {result.njev}")
#     print(f"result status = {result.status}")
#     print(f"result message = {result.message}")
#     print(f"result status = {result.status}")
#     print(f"result active_mask = {result.active_mask}")
#     if result.success:
#         return evaluate_model(freqs, fmin, result.x[0], result.x[1], result.x[2])
#     else:
#         return data

In [ ]:
#print(f"Manual test = {0.95*np.sqrt(psd[-1])}, 30.0, 3.1")
#manual = evaluate_model(freq, 1.0e-5, 0.95*np.sqrt(psd[-1]), 30.0, 3.1)

for freq, psd in zip(test_freq, test_psd):
    fit_psd = fit_log_psd(freq, psd)

    fig = plt.figure(figsize=[12, 8])
    ax = fig.add_subplot(1, 1, 1)
    ax.loglog(
        freq,
        psd,
        color="black",
        label="Input PSD",
    )
#     ax.loglog(
#         freq,
#         manual,
#         color="green",
#         label="Manual",
#     )
    # for fake_net in np.linspace(0.1, 0.127, num=5):
    #     ax.loglog(
    #         freq,
    #         evaluate_model(freq, 1.0e-5, fake_net, 30.0, 3.1),
    #         color="purple",
    #         label="guess",
    #     )
    # for fake_alpha in np.linspace(2.47, 3.1, num=5):
    #     ax.loglog(
    #         freq,
    #         evaluate_model(freq, 1.0e-5, 0.106, 30.0, fake_alpha),
    #         color="purple",
    #         label="guess",
    #     )
    ax.loglog(
        freq,
        fit_psd,
        color="red",
        label="Fit",
    )
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("PSD [K$^2$ / Hz]")
    ax.legend(loc="best")
    plt.show()
